### Лабораторная работа 2. Решение задачи о рюкзаке 0-1.


# Выполнили: Баринов Даниил, Бугров Лев, Голованов Денис

In [25]:
import copy
from queue import PriorityQueue
from tabulate import tabulate

class item():
    def __init__(self, w, p, idx = 0):
        self.w = w
        self.p = p
        self.idx = idx

### Динамическое программирование

In [26]:
def create_tables(items, A, n, W):
    comp_num = 0
    for i in range(0, n+1): 
        A.append([])
        for j in range(0, W+1):
            A[i].append(0)
    for i in range(0, n+1):
        for j in range(0, W+1):
            comp_num +=1
            if i == 0 or j == 0:
                A[i][j] = 0
            elif items[i - 1].w <= j:
                A[i][j] = max(A[i - 1][j], A[i - 1][j - items[i - 1].w] + items[i - 1].p)
            else:
                A[i][j] = A[i - 1][j]
    return comp_num
   

def dynamic(items, W):
    A = []
    k = []
    n = len(items)
    comp_num = create_tables(items, A, n, W)
    for i in range(0, n):
        k.append(0)
    capacity = W + 1
    for i in range(n, 0, -1):
        comp_num += 1
        if A[i][capacity - 1] > A[i - 1][capacity - 1]:
            k[i - 1] = 1
            capacity = capacity - items[i - 1].w

    value = A[n][W]
    return value, k, comp_num


### Метод ветвей и границ

In [27]:
class node():
    def __init__(self, lvl, p, w, ub):
        self.lvl = lvl
        self.p = p
        self.w = w
        self.ub = ub
        
        self.indexes = []

from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

def branch_and_bound(items, W):
    comp_num = 0
    n = len(items)
    items.sort(key = lambda item: item.p / item.w, reverse=True)
    PQ = PriorityQueue()
    PQ.put(PrioritizedItem(-W,node(-1,0,0,0)))
    best_p = 0
    best_x = [0]*n #idk how to correctly save the solution
    
    while (not PQ.empty()):
        u = PQ.get().item
        v = copy.copy(u)
        
        if u.lvl == n - 2:
            continue
        
        #case: x_j = 1
        v.lvl += 1
        v.w += items[v.lvl].w
        v.p += items[v.lvl].p
        
        comp_num += 1
        if (v.w <= W and v.p > best_p):
            best_p = v.p
        
        if v.w < W: 
            v.ub = v.p + (W - v.w)*(items[v.lvl + 1].p / items[v.lvl + 1].w)
        else:
            v.ub = v.p
        
        comp_num += 1
        if (v.ub > best_p):
            PQ.put(PrioritizedItem(-v.ub, copy.copy(v)))
            
        
        #case: x_j = 0
        u.lvl += 1
        
        if u.w < W:
            u.ub = u.p + (W - u.w)*(items[u.lvl + 1].p / items[u.lvl + 1].w)
        else:
            u.ub = u.p
            
        comp_num += 1
        if u.ub > best_p:
            PQ.put(PrioritizedItem(-u.ub, copy.copy(u)))
    return best_p, best_x, comp_num




### 2-approx алгоритм

In [28]:
def greedy_quality(items, W):
    comp_num = 0
    items.sort(key = lambda item: item.p / item.w, reverse=True)
    
    n = len(items)
    j = 0
    sum_p = 0
    sum_w = 0
    x = [0]*n
    
    while j < n and sum_w + items[j].w <= W:
        comp_num += 2
        sum_w += items[j].w
        sum_p += items[j].p
        x[items[j].idx] = 1
        j += 1
        
    return sum_p, x, comp_num


def greedy_profit(items, W):
    comp_num = 0
    items.sort(key = lambda item: item.p, reverse=True)
    
    n = len(items)
    j = 0
    sum_p = 0
    sum_w = 0
    x = [0]*n
    
    while j < n and sum_w + items[j].w <= W:
        comp_num += 2
        sum_w += items[j].w
        sum_p += items[j].p
        x[items[j].idx] = 1
        j += 1
        
    return sum_p, x, comp_num


def approx2(items, W):
    
    greedy_quality_res = greedy_quality(items, W)
    greedy_profit_res = greedy_profit(items, W)
    
    if greedy_profit_res[0] > greedy_quality_res[0]:
        return greedy_profit_res[0], greedy_profit_res[1], greedy_profit_res[2] + greedy_quality_res[2]
    else:
        return greedy_quality_res[0], greedy_quality_res[1],  greedy_profit_res[2] + greedy_quality_res[2]

### PTAS

In [29]:
def get_subsets(arr, k):
    n = len(arr)
    tmp = list(range(k))
    subsets = []
    def get_subsets_rec(idx, a, i):
        if idx == k:
            subsets.append(a.copy())
            return
        if i >= n:
            return
        a[idx] = arr[i]
        get_subsets_rec(idx + 1, a, i + 1)
        get_subsets_rec(idx, a, i + 1) 
        
    get_subsets_rec(0, tmp, 0)
    return subsets

def GS(W, items, subset):
    sum_p = 0
    sum_w = 0
    X = []
    for idx in subset:
        sum_w += items[idx].w
    available_w = W - sum_w
    
    for idx in range(len(items)):
        if idx not in subset and items[idx].w <= available_w:
            sum_p += items[idx].p
            available_w -= items[idx].w
            X.append(idx)

    return sum_p, X

def ptas(items, W, k=2):
    comp_num = 0
    indexes = list(range(len(items)))
    subsets = get_subsets(indexes, k)
    best_p = 0
    best_x = []
    
    for subset in subsets:
        subset_w = 0
        subset_p = 0
        for idx in subset:
            subset_w += items[idx].w
            subset_p += items[idx].p

        comp_num += 1
        if subset_w <= W:
            p, X = GS(W, items, subset)
            comp_num += 1
            if p + subset_p > best_p:
                best_p = p + subset_p
                best_x = list(set(X) | set(subset))  
    return best_p, best_x, comp_num



### Тесты, бенчмарки

In [30]:
import os

def create_data(path, number):
    weights = []
    values = []
    answers = []
    W = 0
    for filename in sorted(os.listdir(path)):
        inp = open(path + '/' + filename).read()
        if str(number) in filename:
            if "_c" in filename:
                W = int(inp)
            elif "_p" in filename:
                values = list(map(int, inp.split()))
            elif "_s" in filename:
                answers = list(map(int, inp.split()))
            elif "_w" in filename:
                weights = list(map(int, inp.split()))  
    items = []
    for i in range(len(weights)):
        items.append(item(weights[i], values[i], i))
    
    return items, answers, W

In [31]:
import time

def tests(alg):
    for i in range(1, 7):
        items,answers,W = create_data('benchmarks', i)
        
        correct_value = 0
        for i in range(len(answers)):
            correct_value += answers[i]*items[i].p
        print(f'Correct value: {correct_value}')
        
        solution = alg(items, W)[0]
        print(f'Value {solution}')
        assert solution >= correct_value / 2 and solution <= correct_value
        print('Success')

            
def measure_algorithm_time(alg, items, W):
    start_time = time.time()
    for i in range(100):
        solution = alg(items, W)
    end_time = time.time()
    return round((end_time - start_time)/100, 5), solution[2]


def compute_results():
    results = {'benchmark_1':[], 'benchmark_2':[],'benchmark_3':[], 'benchmark_4':[],'benchmark_5':[], 'benchmark_6':[],'benchmark_7':[]}
    
    for i in range(1, 8):
        items,answers,W = create_data('benchmarks', i)
        
        results[f'benchmark_{i}'].append(measure_algorithm_time(dynamic, items, W))
        results[f'benchmark_{i}'].append(measure_algorithm_time(branch_and_bound, items, W))
        results[f'benchmark_{i}'].append(measure_algorithm_time(approx2, items, W))
        results[f'benchmark_{i}'].append(measure_algorithm_time(ptas, items, W))
    return results

In [32]:
def display_table_of_results(results):
    table_time = [[' ', 'Dynamic', 'Branch and Bounds', '2-approximation' ,'PTAS']]
    
    for benchmark in results:
        table_time.append([benchmark] + list(map(lambda x: f'{x[0]} / {x[1]}', results[benchmark])))
    
    print('Measurments of running time of algorithms and the number of comparisons (time, s / comparions number)')
    print(tabulate(table_time, tablefmt='fancy_grid'))

In [33]:
res = compute_results()

In [34]:
display_table_of_results(res)

Measurments of running time of algorithms and the number of comparisons (time, s / comparions number)
╒═════════════╤═════════════════╤═══════════════════╤═════════════════╤═══════════════╕
│             │ Dynamic         │ Branch and Bounds │ 2-approximation │ PTAS          │
├─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────┤
│ benchmark_1 │ 0.00098 / 1836  │ 0.00066 / 96      │ 1e-05 / 12      │ 0.00014 / 87  │
├─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────┤
│ benchmark_2 │ 8e-05 / 167     │ 0.00017 / 33      │ 0.0 / 8         │ 3e-05 / 20    │
├─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────┤
│ benchmark_3 │ 0.0004 / 1343   │ 0.00039 / 63      │ 0.0 / 8         │ 5e-05 / 30    │
├─────────────┼─────────────────┼───────────────────┼─────────────────┼───────────────┤
│ benchmark_4 │ 0.00021 / 415   │ 0.00058 / 75      │ 1e-05 / 6       │ 6e-05 / 41    │
├─────────────┼───

In [35]:
tests(dynamic)

Correct value: 309
Value 309
Success
Correct value: 51
Value 51
Success
Correct value: 150
Value 150
Success
Correct value: 107
Value 107
Success
Correct value: 900
Value 900
Success
Correct value: 1735
Value 1735
Success


In [36]:
tests(approx2)

Correct value: 309
Value 266
Success
Correct value: 51
Value 47
Success
Correct value: 150
Value 114
Success
Correct value: 107
Value 90
Success
Correct value: 900
Value 850
Success
Correct value: 1735
Value 1478
Success


In [37]:
tests(branch_and_bound)

Correct value: 309
Value 309
Success
Correct value: 51
Value 51
Success
Correct value: 150
Value 150
Success
Correct value: 107
Value 107
Success
Correct value: 900
Value 898
Success
Correct value: 1735
Value 1735
Success


In [38]:
tests(ptas)

Correct value: 309
Value 309
Success
Correct value: 51
Value 51
Success
Correct value: 150
Value 150
Success
Correct value: 107
Value 107
Success
Correct value: 900
Value 900
Success
Correct value: 1735
Value 1652
Success
